In [1]:
from langchain.chat_models.openai import ChatOpenAI
from langchain.output_parsers import PydanticOutputParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os

In [2]:
# Custom imports:
from content_collection import collect_serp_data_and_extract_text_from_webpages
from custom_summarize_chain import create_all_summaries, DocumentSummary
from expert_interview_chain import InterviewChain

## Topic Research:

In [3]:
%pip install google-search-results pandas html2text pytest-playwright chromadb nest_asyncio --quiet

Note: you may need to restart the kernel to use updated packages.


In [4]:
!playwright install

In [5]:
import nest_asyncio
nest_asyncio.apply()

In [6]:
# Constant variables:
TOPIC = "Neural networks"
os.environ["SERPAPI_API_KEY"] = ""

In [8]:
# Extract content from webpages into LangChain documents:
text_documents = await collect_serp_data_and_extract_text_from_webpages(topic=TOPIC)

/var/folders/_y/20jl658s4jl0zvy5c0x0c5140000gn/T/ipykernel_59331/796634559.py:2: RuntimeWarning: coroutine 'collect_serp_data_and_extract_text_from_webpages' was never awaited
  text_documents = await collect_serp_data_and_extract_text_from_webpages(topic=TOPIC)


In [10]:
# LLM, text splitter + parser:
llm = ChatOpenAI(temperature=0)
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=1500, chunk_overlap=400
)
parser = PydanticOutputParser(pydantic_object=DocumentSummary)

In [11]:
summaries = await create_all_summaries(text_documents, parser, llm, text_splitter)

Summarizing the data!
Summarizing the data!
Summarizing the data!


---

## Expert Interview Questions:


In [15]:
interview_chain = InterviewChain(topic=TOPIC, document_summaries=summaries)
interview_questions = interview_chain()

----------------------------------------

## Answer The Interview Questions:

In [29]:
for question in interview_questions.questions:
    print(f"Answer the following question: {question.question}\n", flush=True)
    answer = input(f"Answer the following question: {question.question}\n")
    print('------------------------------------------')
    question.answer = answer

Answer the following question: Can you explain the concept of backpropagation and its role in training neural networks?

------------------------------------------
Answer the following question: What are some of the challenges or limitations of neural networks?

------------------------------------------
Answer the following question: How do neural networks differ from other machine learning algorithms?

------------------------------------------
Answer the following question: What are some real-world applications of neural networks?

------------------------------------------
Answer the following question: What are the current trends or advancements in neural network research?

------------------------------------------


---

## General Article Outline:


In [37]:
from article_outline_generation import BlogOutlineGenerator
blog_outline_generator = BlogOutlineGenerator(topic=TOPIC, questions_and_answers=[item.dict()  
                                                                                  for item in interview_questions.questions ] )
questions_and_answers = blog_outline_generator.questions_and_answers
outline_result = blog_outline_generator.generate_outline(summaries)

---

## Article Text Generation:


In [39]:
from article_generation import ContentGenerator
content_gen = ContentGenerator(topic=TOPIC, outline=outline_result, questions_and_answers=questions_and_answers)
content_gen.split_and_vectorize_documents(text_documents)
blog_post = content_gen.generate_blog_post()

---

## Title Tag Optimization:


In [ ]:
%pip install thumb --quiet # This is a tool for evaluation of the potential blog post titles.

Note: you may need to restart the kernel to use updated packages.


In [59]:
import os
import thumb

# Set the environment variables (get your API key: https://platform.openai.com/account/api-keys)
os.environ["LANGCHAIN_API_KEY"] = "YOUR_API_KEY_HERE" # optional: for langsmith tracing

In [66]:
base_prompt = f"Generate a title tag for this blog post on topic: {TOPIC}, Title: {outline_result.title}. The first 200 characters: {''.join(blog_post)[0:200]}"

# Set up a prompt templates for the a/b test:
prompt_a = base_prompt + ' ' + "Write the title in a style of {style}. Make sure that it is SEO friendly."
prompt_b = base_prompt + ' ' + "Write the title in a style of {style}. Make sure that is reader friendly. Focus on the reader."

# Set test cases with different input variables:
cases = [
  {"style": "business professional"},
  {"style": "b2b marketing"},
  ]

# Generate the responses
test = thumb.test([prompt_a, prompt_b], cases, runs=10)

HTML(value='"Unlock the Power of Neural Networks: A Comprehensive Guide for Enhanced Pattern Recognition and D…

IntProgress(value=0, description='Progress:', max=40)

Label(value='ThumbTest: 80615803')